In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from datetime import datetime

date_ = datetime.utcnow().strftime("%Y-%m-%d")
silver_path = f"abfss://projectfiles@practicestorageacc0.dfs.core.windows.net/githublogs/silver/{date_}"
gold_folder = f"abfss://projectfiles@practicestorageacc0.dfs.core.windows.net/githublogs/gold/"


df_silver = spark.readStream.format("delta").load(silver_path)

In [0]:
df_silver.display()

timestamp event_type repo actor raw event_time org repo_name 2025-07-15T21:06:05Z pushevent AS271/weathermaps scripts-nw {"id": "52128701382", "type": "PushEvent", "actor": {"id": 210576632, "login": "scripts-nw", "display_login": "scripts-nw", "gravatar_id": "", "url": "https://api.github.com/users/scripts-nw", "avatar_url": "https://avatars.githubusercontent.com/u/210576632?"}, "repo": {"id": 978864437, "name": "AS271/weathermaps", "url": "https://api.github.com/repos/AS271/weathermaps"}, "payload": {"repository_id": 978864437, "push_id": 25517538400, "size": 1, "distinct_size": 1, "ref": "refs/heads/main", "head": "8207a09349ce76c750d4fd285a807b082e77d3cb", "before": "7116a6a7e4eee1f469024406bfca88aba8c05c6e", "commits": [{"sha": "8207a09349ce76c750d4fd285a807b082e77d3cb", "author": {"email": "scripts_nw@bc.net", "name": "scripts_nw@bc.net"}, "message": "latest weathermap", "distinct": true, "url": "https://api.github.com/repos/AS271/weathermaps/commits/8207a09349ce76c750d4fd285a807b082e77d3cb"}]}, "public": true, "created_at": "2025-07-15T21:06:05Z", "org": {"id": 46945828, "login": "AS271", "gravatar_id": "", "url": "https://api.github.com/orgs/AS271", "avatar_url": "https://avatars.githubusercontent.com/u/46945828?"}} 2025-07-15T21:06:05.000Z AS271 weathermaps 2025-07-15T21:06:05Z pullrequestevent parhelia512/InvokeAI pull[bot] {"id": "52128701365", "type": "PullRequestEvent", "actor": {"id": 39814207, "login": "pull[bot]", "display_login": "pull", "gravatar_id": "", "url": "https://api.github.com/users/pull[bot]", "avatar_url": "https://avatars.githubusercontent.com/u/39814207?"}, "repo": {"id": 665087685, "name": "parhelia512/InvokeAI", "url": "https://api.github.com/repos/parhelia512/InvokeAI"}, "payload": {"action": "opened", "number": 75, "pull_request": {"url": "https://api.github.com/repos/parhelia512/InvokeAI/pulls/75", "id": 2668691547, "node_id": "PR_kwDOJ6Ruxc6fEQBb", "html_url": "https://github.com/parhelia512/InvokeAI/pull/75", "diff_url": "https://github.com/parhelia512/InvokeAI/pull/75.diff", "patch_url": "https://github.com/parhelia512/InvokeAI/pull/75.patch", "issue_url": "https://api.github.com/repos/parhelia512/InvokeAI/issues/75", "number": 75, "state": "open", "locked": true, "title": "[pull] main from invoke-ai:main", "user": {"login": "pull[bot]", "id": 39814207, "node_id": "MDM6Qm90Mzk4MTQyMDc=", "avatar_url": "https://avatars.githubusercontent.com/in/12910?v=4", "gravatar_id": "", "url": "https://api.github.com/users/pull%5Bbot%5D", "html_url": "https://github.com/apps/pull", "followers_url": "https://api.github.com/users/pull%5Bbot%5D/followers", "following_url": "https://api.github.com/users/pull%5Bbot%5D/following{/other_user}", "gists_url": "https://api.github.com/users/pull%5Bbot%5D/gists{/gist_id}", "starred_url": "https://api.github.com/users/pull%5Bbot%5D/starred{/owner}{/repo}", "subscriptions_url": "https://api.github.com/users/pull%5Bbot%5D/subscriptions", "organizations_url": "https://api.github.com/users/pull%5Bbot%5D/orgs", "repos_url": "https://api.github.com/users/pull%5Bbot%5D/repos", "events_url": "https://api.github.com/users/pull%5Bbot%5D/events{/privacy}", "received_events_url": "https://api.github.com/users/pull%5Bbot%5D/received_events", "type": "Bot", "user_view_type": "public", "site_admin": false}, "body": "See Commits and Changes for more details.\n\n-----\nCreated by [ **pull[bot]**](https://github.com/wei/pull) (v2.0.0-alpha.3)\n\n_Can you help keep this open source service alive? **[\ud83d\udc96 Please sponsor : )](https://prod.download/pull-pr-sponsor)**_", "created_at": "2025-07-15T21:06:03Z", "updated_at": "2025-07-15T21:06:04Z", "closed_at": null, "merged_at": null, "merge_commit_sha": "2dd6d272b4656f0c9fc8246fbc774e200c657edc", "assignee": null, "assignees": [], "requested_reviewers": [], "requested_teams": [], "labels": [], "milestone": null, "draft": false, "commits_url": "https://api.github.com/repos/parhelia512/InvokeAI/pulls/75/commits", "review_comments_ur

In [0]:
df_event_counts = df_silver \
    .withWatermark("event_time", "2 minutes")\
    .groupBy(
        window("event_time", "1 minute"),
        col("event_type")
    ).agg(count("*").alias("event_count"))

In [0]:
df_active_users = df_silver \
    .withWatermark("event_time", "2 hours")\
    .groupBy(
        window("event_time", "1 hour"),
        col("org")
    ).agg(approx_count_distinct("actor").alias("active_users")) # streaming does not support countDistinct, which requires fulll shuffling, so we need to use approx_count_distinct


In [0]:
df_top_repos = df_silver \
    .withWatermark("event_time", "10 minutes") \
    .groupBy(
        window("event_time", "5 minutes"),
        col("repo")
    ).agg(count("*").alias("activity_count"))

In [0]:
events_path = gold_folder + "events/" + date_
active_users_path = gold_folder + "active_users/" + date_
top_repos_path = gold_folder + "repos_activity/" + date_

df_event_counts.writeStream \
    .format("delta") \
    .option("checkpointLocation", f"/temp/checkpoints/githublogs/gold/event/{date_}") \
    .outputMode("append") \
    .option("path", events_path) \
    .trigger(availableNow=True) \
    .start()

df_active_users.writeStream \
    .format("delta") \
    .option("checkpointLocation", f"/temp/checkpoints/githublogs/gold/users/{date_}") \
    .outputMode("append") \
    .option("path", active_users_path) \
    .trigger(availableNow=True) \
    .start()


df_top_repos.writeStream \
    .format("delta") \
    .option("checkpointLocation", f"/temp/checkpoints/githublogs/gold/repo/{date_}") \
    .outputMode("append") \
    .option("path", top_repos_path) \
    .trigger(availableNow=True) \
    .start()

